# Simulation of brownian particles through a pore

## Preamble

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from brownpy import Universe
from brownpy.topology import ElasticPore1

## Define simulation setup

In [ ]:
N_particles = 8*1024 # Number of particles (power of 2 is better for gpu)
D = 1.5E-4 # A²/fs  (1.5E-9 m²/s) - Diffusion coefficient
dt = 1e6 # fs (1ns)
seed = 1789 # Specify seed for reproductibility
# Define geometry
R = 10 # A (1nm) radius of the pore
Lm = 1000 # A (100nm) distance between each pores
L = 1E4 # A (1um) reservoir depth 
top = ElasticPore1(Lm, L, R, seed=seed) # Just an empty space
# Define simulation setup
u = Universe(top, N_particles, D, dt,
             output_path='empty', overwrite=True)

## Plotting initial configuration

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,4))
u.plot(axes[0])
u.plot(axes[1])
axes[1].set_xlim(-300, 300)
axes[1].set_ylim(-300, 300)

## Watching particle trajectory

In [ ]:
# Be carefull ! Here it will already generate 68MB of data !
u.run(10_000, freq_dumps=10)

In [ ]:
# Retrieving trajectory from output file and plot it
traj = u.f['/run/0/trajectory']
x = traj[:,0]
# Getting index of particles that go through the pore
temp = np.argsort(np.sum(x<0, axis=1)*np.sum(x>0, axis=1))[::-1] 
fig, ax = plt.subplots()
top.plot(ax)
i_particle = temp[1]
x, y = traj[i_particle,0,:], traj[i_particle,1,:]
ax.scatter(x, y, c=np.arange(traj.shape[2]), marker='.', cmap='jet', s=4)
ax.set_aspect('equal')
ax.set_xlim(x.min(), x.max())

## Show number of particles in specific region of space

In [ ]:
# Since we do not record trajectory we can make a longer simulation
u.run(1_000_000)

In [ ]:
# In-built topology has regions already defined by defaut.
# However, you can specify it in 'run' function argument 
top.regions

In [ ]:
# Let's plot it
left = u.f['/run/1/regions/left']
plt.plot(left)